In [1]:
from handler import get_current_data

current_price, close, ts = get_current_data()

In [2]:
print(f"current price: {current_price} close: {close[-1]}")

current price: 104.89 close: 104.88999938964844


In [3]:
from handler import calculate_bbands_series
import pandas as pd

bands = calculate_bbands_series(
    prices=close,
    timestamps=ts,
)
df = pd.DataFrame(bands)    

In [9]:
# Add this cell after creating your DataFrame
print("Date range in data:")
print("Start:", pd.to_datetime(df['timestamp'].min(), unit='s'))
print("End:", pd.to_datetime(df['timestamp'].max(), unit='s'))

Date range in data:
Start: 2009-09-25 07:00:00
End: 2024-11-22 08:00:00


In [8]:
df.tail()

,middle,upper,lower,timestamp,price
3851,100.5780,104.668891,96.487109,1731916800,102.379997
3852,100.6995,104.842117,96.556883,1732003200,102.269997
3853,100.8425,105.009608,96.675391,1732089600,102.370003
3854,101.0575,105.357188,96.757812,1732176000,103.720001
3855,101.3145,105.890692,96.738308,1732262400,104.889999


In [13]:
# First convert timestamps to milliseconds (Bokeh expects milliseconds)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df['timestamp'] = df['timestamp'].astype('int64') // 10**6  # Convert to milliseconds

# Create the Bokeh figure
p = figure(
    width=1200,
    height=800,
    x_axis_type='datetime',  # This tells Bokeh to expect milliseconds timestamps
    title='SWDA.MI ETF with Bollinger Bands',
    tools='pan,box_zoom,wheel_zoom,reset,save',
    active_scroll='wheel_zoom'
)

# Rest of your plotting code remains the same
p.line(df['timestamp'], df['price'], line_color='black', legend_label='Price', line_width=2)
p.line(df['timestamp'], df['upper'], line_color='red', legend_label='Upper Band', line_width=1)
p.line(df['timestamp'], df['lower'], line_color='green', legend_label='Lower Band', line_width=1)
p.line(df['timestamp'], df['middle'], line_color='blue', legend_label='20-day MA', line_width=1)

# Add red dots for points below lower band
below_lb_mask = df['price'] < df['lower']
p.circle(df['timestamp'][below_lb_mask], df['price'][below_lb_mask], 
         size=8, color='red', legend_label='Below Lower Band')

# For the patch, we need to handle the timestamps the same way
band_x = np.append(df['timestamp'], df['timestamp'][::-1])
band_y = np.append(df['upper'], df['lower'][::-1])
p.patch(band_x, band_y, color='gray', alpha=0.2, legend_label='Bollinger Bands')

# Customize the plot
p.grid.grid_line_alpha = 0.3
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Add axis labels
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'

# Show the plot
show(p)